# การปรับแต่งโมเดล Open AI

สมุดบันทึกนี้อ้างอิงจากคำแนะนำล่าสุดในเอกสาร [Fine Tuning](https://platform.openai.com/docs/guides/fine-tuning?WT.mc_id=academic-105485-koreyst) ของ Open AI

การปรับแต่งโมเดล (Fine-tuning) จะช่วยเพิ่มประสิทธิภาพของโมเดลพื้นฐานให้เหมาะกับแอปพลิเคชันของคุณมากขึ้น โดยการฝึกโมเดลใหม่ด้วยข้อมูลและบริบทเพิ่มเติมที่เกี่ยวข้องกับกรณีการใช้งานนั้นๆ โปรดทราบว่าเทคนิคการออกแบบ prompt เช่น _few shot learning_ และ _retrieval augmented generation_ จะช่วยให้คุณสามารถเสริมข้อมูลที่เกี่ยวข้องเข้าไปใน prompt เพื่อยกระดับคุณภาพได้ อย่างไรก็ตาม วิธีเหล่านี้จะถูกจำกัดด้วยขนาดหน้าต่างโทเคนสูงสุดของโมเดลพื้นฐานที่เลือกใช้

แต่เมื่อใช้การปรับแต่งโมเดล เราจะฝึกโมเดลใหม่โดยตรงด้วยข้อมูลที่ต้องการ (ทำให้สามารถใช้ตัวอย่างได้มากกว่าที่จะใส่ในหน้าต่างโทเคนสูงสุด) และสามารถนำโมเดลเวอร์ชัน _custom_ ที่ผ่านการฝึกมาใช้งานได้ทันทีโดยไม่ต้องใส่ตัวอย่างในขั้นตอน inference อีกต่อไป วิธีนี้ไม่เพียงแต่ช่วยให้การออกแบบ prompt มีประสิทธิภาพมากขึ้น (เพราะเราสามารถใช้หน้าต่างโทเคนกับสิ่งอื่นได้ยืดหยุ่นขึ้น) แต่ยังอาจช่วยลดค่าใช้จ่าย (เพราะใช้โทเคนส่งเข้าโมเดลน้อยลงในแต่ละครั้งที่ใช้งาน)

ขั้นตอนของการปรับแต่งโมเดลมี 4 ขั้นตอนหลัก:
1. เตรียมข้อมูลสำหรับฝึกและอัปโหลดขึ้นระบบ
1. รันงานฝึกโมเดลเพื่อให้ได้โมเดลที่ปรับแต่งแล้ว
1. ประเมินผลโมเดลที่ปรับแต่งแล้วและปรับปรุงคุณภาพตามต้องการ
1. นำโมเดลที่ปรับแต่งแล้วไปใช้งานจริงเมื่อพอใจกับผลลัพธ์

โปรดทราบว่าไม่ใช่ทุกโมเดลพื้นฐานจะรองรับการปรับแต่ง ตรวจสอบข้อมูลล่าสุดได้ที่ [เอกสาร OpenAI](https://platform.openai.com/docs/guides/fine-tuning/what-models-can-be-fine-tuned?WT.mc_id=academic-105485-koreyst) คุณยังสามารถปรับแต่งโมเดลที่เคยปรับแต่งมาก่อนแล้วได้อีกด้วย ในบทเรียนนี้ เราจะใช้ `gpt-35-turbo` เป็นโมเดลพื้นฐานเป้าหมายสำหรับการปรับแต่ง

---


### ขั้นตอนที่ 1.1: เตรียมชุดข้อมูลของคุณ

เราจะสร้างแชทบอทที่ช่วยให้คุณเข้าใจตารางธาตุ โดยตอบคำถามเกี่ยวกับธาตุแต่ละตัวด้วยกลอนลิเมอริก ในบทเรียนง่ายๆ นี้ เราจะสร้างชุดข้อมูลเพื่อฝึกโมเดล โดยมีตัวอย่างคำตอบไม่กี่ตัวอย่างที่แสดงรูปแบบข้อมูลที่ต้องการ ในการใช้งานจริง คุณจะต้องสร้างชุดข้อมูลที่มีตัวอย่างมากกว่านี้ หรืออาจใช้ชุดข้อมูลเปิด (สำหรับโดเมนของคุณ) ถ้ามีอยู่ แล้วปรับรูปแบบให้เหมาะกับการปรับแต่งโมเดล

เนื่องจากเราจะใช้ `gpt-35-turbo` และต้องการคำตอบแบบรอบเดียว (chat completion) เราสามารถสร้างตัวอย่างโดยใช้ [รูปแบบที่แนะนำนี้](https://platform.openai.com/docs/guides/fine-tuning/preparing-your-dataset?WT.mc_id=academic-105485-koreyst) ซึ่งตรงกับข้อกำหนดของ OpenAI chat completion ถ้าคุณต้องการเนื้อหาการสนทนาแบบหลายรอบ ให้ใช้ [รูปแบบตัวอย่างหลายรอบ](https://platform.openai.com/docs/guides/fine-tuning/multi-turn-chat-examples?WT.mc_id=academic-105485-koreyst) ซึ่งจะมีพารามิเตอร์ `weight` เพื่อระบุว่าข้อความไหนควรใช้ (หรือไม่ใช้) ในกระบวนการปรับแต่ง

สำหรับบทเรียนนี้ เราจะใช้รูปแบบรอบเดียวที่ง่ายกว่า ข้อมูลจะอยู่ใน [รูปแบบ jsonl](https://jsonlines.org/?WT.mc_id=academic-105485-koreyst) โดยมี 1 เรคคอร์ดต่อบรรทัด แต่ละเรคคอร์ดเป็นอ็อบเจ็กต์ในรูปแบบ JSON ตัวอย่างด้านล่างแสดง 2 เรคคอร์ดเป็นตัวอย่าง - ดู [training-data.jsonl](../../../../../18-fine-tuning/python/openai/training-data.jsonl) สำหรับชุดตัวอย่างเต็ม (10 ตัวอย่าง) ที่เราจะใช้ในบทเรียนปรับแต่งโมเดล **หมายเหตุ:** แต่ละเรคคอร์ด _ต้อง_ อยู่ในบรรทัดเดียว (ห้ามแบ่งหลายบรรทัดเหมือนไฟล์ JSON ที่จัดรูปแบบทั่วไป)

```json
{ "messages": [{"role": "system", "content": "Elle is a factual chatbot that answers questions about elements in the periodic table with a limerick"}, {"role": "user", "content": "Tell me about Gallium"}, {"role": "assistant", "content": "Gallium, oh gallium, so light - Melts in your hand, oh what a sight - At 86 degrees - Its liquid with ease - And in semiconductors, it's out of sight"}]}
{ "messages": [{"role": "system", "content": "Elle is a factual chatbot that answers questions about elements in the periodic table with a limerick"}, {"role": "user", "content": "Tell me about Hydrogen"}, {"role": "assistant", "content": "Hydrogen, the first in the line - The lightest of all, so divine - It's in water, you see - And in stars, it's the key - The universe's most common sign"}]}
```

ในการใช้งานจริง คุณจะต้องมีชุดตัวอย่างที่ใหญ่กว่านี้เพื่อให้ได้ผลลัพธ์ที่ดี โดยต้องแลกกับคุณภาพของคำตอบและเวลา/ค่าใช้จ่ายในการปรับแต่งโมเดล เราใช้ชุดตัวอย่างเล็กเพื่อให้ปรับแต่งได้เร็วและเห็นกระบวนการชัดเจน ดู [ตัวอย่างใน OpenAI Cookbook นี้](https://github.com/openai/openai-cookbook/blob/main/examples/How_to_finetune_chat_models.ipynb?WT.mc_id=academic-105485-koreyst) สำหรับบทเรียนปรับแต่งโมเดลที่ซับซ้อนมากขึ้น


### ขั้นตอนที่ 1.2 อัปโหลดชุดข้อมูลของคุณ

อัปโหลดข้อมูลโดยใช้ Files API [ตามที่อธิบายไว้ที่นี่](https://platform.openai.com/docs/guides/fine-tuning/upload-a-training-file) โปรดทราบว่าในการรันโค้ดนี้ คุณต้องดำเนินการตามขั้นตอนต่อไปนี้ก่อน:
 - ติดตั้งแพ็กเกจ Python `openai` (ตรวจสอบให้แน่ใจว่าใช้เวอร์ชัน >=0.28.0 เพื่อให้ได้ฟีเจอร์ล่าสุด)
 - ตั้งค่าตัวแปรสภาพแวดล้อม `OPENAI_API_KEY` ให้เป็นคีย์ OpenAI API ของคุณ
หากต้องการเรียนรู้เพิ่มเติม ดู [คู่มือการตั้งค่า](./../../../00-course-setup/02-setup-local.md?WT.mc_id=academic-105485-koreyst) ที่เตรียมไว้สำหรับคอร์สนี้

ตอนนี้ ให้รันโค้ดเพื่อสร้างไฟล์สำหรับอัปโหลดจากไฟล์ JSONL ในเครื่องของคุณ


In [24]:
from openai import OpenAI
client = OpenAI()

ft_file = client.files.create(
  file=open("./training-data.jsonl", "rb"),
  purpose="fine-tune"
)

print(ft_file)
print("Training File ID: " + ft_file.id)

FileObject(id='file-JdAJcagdOTG6ACNlFWzuzmyV', bytes=4021, created_at=1715566183, filename='training-data.jsonl', object='file', purpose='fine-tune', status='processed', status_details=None)
Training File ID: file-JdAJcagdOTG6ACNlFWzuzmyV


### ขั้นตอนที่ 2.1: สร้างงาน Fine-tuning ด้วย SDK


In [25]:
from openai import OpenAI
client = OpenAI()

ft_filejob = client.fine_tuning.jobs.create(
  training_file=ft_file.id, 
  model="gpt-3.5-turbo"
)

print(ft_filejob)
print("Fine-tuning Job ID: " + ft_filejob.id)

FineTuningJob(id='ftjob-Usfb9RjasncaZ5Cjbuh1XSCh', created_at=1715566184, error=Error(code=None, message=None, param=None), fine_tuned_model=None, finished_at=None, hyperparameters=Hyperparameters(n_epochs='auto', batch_size='auto', learning_rate_multiplier='auto'), model='gpt-3.5-turbo-0125', object='fine_tuning.job', organization_id='org-EZ6ag0n0S6Zm8eV9BSWKmE6l', result_files=[], seed=830529052, status='validating_files', trained_tokens=None, training_file='file-JdAJcagdOTG6ACNlFWzuzmyV', validation_file=None, estimated_finish=None, integrations=[], user_provided_suffix=None)
Fine-tuning Job ID: ftjob-Usfb9RjasncaZ5Cjbuh1XSCh


### ขั้นตอนที่ 2.2: ตรวจสอบสถานะของงาน

นี่คือตัวอย่างสิ่งที่คุณสามารถทำได้ด้วย API `client.fine_tuning.jobs`:
- `client.fine_tuning.jobs.list(limit=<n>)` - แสดงรายการงาน fine-tuning ล่าสุด n งาน
- `client.fine_tuning.jobs.retrieve(<job_id>)` - ดูรายละเอียดของงาน fine-tuning เฉพาะเจาะจง
- `client.fine_tuning.jobs.cancel(<job_id>)` - ยกเลิกงาน fine-tuning
- `client.fine_tuning.jobs.list_events(fine_tuning_job_id=<job_id>, limit=<b>)` - แสดงรายการเหตุการณ์สูงสุด n รายการจากงาน
- `client.fine_tuning.jobs.create(model="gpt-35-turbo", training_file="your-training-file.jsonl", ...)`

ขั้นตอนแรกของกระบวนการนี้คือ _ตรวจสอบความถูกต้องของไฟล์ฝึกสอน_ เพื่อให้แน่ใจว่าข้อมูลอยู่ในรูปแบบที่ถูกต้อง


In [26]:
from openai import OpenAI
client = OpenAI()

# List 10 fine-tuning jobs
client.fine_tuning.jobs.list(limit=10)

# Retrieve the state of a fine-tune
client.fine_tuning.jobs.retrieve(ft_filejob.id)

# List up to 10 events from a fine-tuning job
client.fine_tuning.jobs.list_events(fine_tuning_job_id=ft_filejob.id, limit=10)

SyncCursorPage[FineTuningJobEvent](data=[FineTuningJobEvent(id='ftevent-GkWiDgZmOsuv4q5cSTEGscY6', created_at=1715566184, level='info', message='Validating training file: file-JdAJcagdOTG6ACNlFWzuzmyV', object='fine_tuning.job.event', data={}, type='message'), FineTuningJobEvent(id='ftevent-3899xdVTO3LN7Q7LkKLMJUnb', created_at=1715566184, level='info', message='Created fine-tuning job: ftjob-Usfb9RjasncaZ5Cjbuh1XSCh', object='fine_tuning.job.event', data={}, type='message')], object='list', has_more=False)

In [30]:
# Once the training data is validated
# Track the job status to see if it is running and when it is complete
from openai import OpenAI
client = OpenAI()

response = client.fine_tuning.jobs.retrieve(ft_filejob.id)

print("Job ID:", response.id)
print("Status:", response.status)
print("Trained Tokens:", response.trained_tokens)

Job ID: ftjob-Usfb9RjasncaZ5Cjbuh1XSCh
Status: running
Trained Tokens: None


### ขั้นตอนที่ 2.3: ติดตามเหตุการณ์เพื่อตรวจสอบความคืบหน้า


In [44]:
# You can also track progress in a more granular way by checking for events
# Refresh this code till you get the `The job has successfully completed` message
response = client.fine_tuning.jobs.list_events(ft_filejob.id)

events = response.data
events.reverse()

for event in events:
    print(event.message)

Step 85/100: training loss=0.14
Step 86/100: training loss=0.00
Step 87/100: training loss=0.00
Step 88/100: training loss=0.07
Step 89/100: training loss=0.00
Step 90/100: training loss=0.00
Step 91/100: training loss=0.00
Step 92/100: training loss=0.00
Step 93/100: training loss=0.00
Step 94/100: training loss=0.00
Step 95/100: training loss=0.08
Step 96/100: training loss=0.05
Step 97/100: training loss=0.00
Step 98/100: training loss=0.00
Step 99/100: training loss=0.00
Step 100/100: training loss=0.00
Checkpoint created at step 80 with Snapshot ID: ft:gpt-3.5-turbo-0125:bitnbot::9OFWyyF2:ckpt-step-80
Checkpoint created at step 90 with Snapshot ID: ft:gpt-3.5-turbo-0125:bitnbot::9OFWyzhK:ckpt-step-90
New fine-tuned model created: ft:gpt-3.5-turbo-0125:bitnbot::9OFWzNjz
The job has successfully completed


### ขั้นตอนที่ 2.4: ดูสถานะในแดชบอร์ดของ OpenAI


คุณยังสามารถดูสถานะได้โดยเข้าไปที่เว็บไซต์ OpenAI แล้วเลือกส่วน _Fine-tuning_ บนแพลตฟอร์ม ตรงนี้จะบอกสถานะของงานที่กำลังดำเนินการอยู่ และยังให้คุณติดตามประวัติการรันงานก่อนหน้านี้ได้ด้วย ในภาพหน้าจอนี้ คุณจะเห็นว่าการรันครั้งก่อนล้มเหลว และการรันครั้งที่สองสำเร็จ สำหรับบริบทนี้ เกิดขึ้นเมื่อการรันครั้งแรกใช้ไฟล์ JSON ที่มีข้อมูลไม่ถูกต้อง - เมื่อแก้ไขแล้ว การรันครั้งที่สองก็สำเร็จและทำให้โมเดลพร้อมใช้งาน


คุณยังสามารถดูข้อความสถานะและตัวชี้วัดได้โดยเลื่อนลงไปที่แดชบอร์ดภาพตามที่แสดงด้านล่างนี้:

| ข้อความ | ตัวชี้วัด |
|:---|:---|
| ![Messages](../../../../../translated_images/fine-tuned-messages-panel.4ed0c2da5ea1313b3a706a66f66bf5007c379cd9219cfb74cb30c0b04b90c4c8.th.png) |  ![Metrics](../../../../../translated_images/fine-tuned-metrics-panel.700d7e4995a652299584ab181536a6cfb67691a897a518b6c7a2aa0a17f1a30d.th.png)|


### ขั้นตอนที่ 3.1: ดึงรหัส ID & ทดสอบโมเดลที่ปรับแต่งแล้วในโค้ด


In [46]:
# Retrieve the identity of the fine-tuned model once ready
response = client.fine_tuning.jobs.retrieve(ft_filejob.id)
fine_tuned_model_id = response.fine_tuned_model
print("Fine-tuned Model ID:", fine_tuned_model_id)

Fine-tuned Model ID: ft:gpt-3.5-turbo-0125:bitnbot::9OFWzNjz


In [47]:
# You can then use that model to generate completions from the SDK as shown
# Or you can load that model into the OpenAI Playground (in the UI) to validate it from there.
from openai import OpenAI
client = OpenAI()

completion = client.chat.completions.create(
  model=fine_tuned_model_id,
  messages=[
    {"role": "system", "content": "You are Elle, a factual chatbot that answers questions about elements in the periodic table with a limerick"},
    {"role": "user", "content": "Tell me about Strontium"},
  ]
)
print(completion.choices[0].message)

ChatCompletionMessage(content="Strontium, a metal so bright - It's in fireworks, a dazzling sight - It's in bones, you see - And in tea, it's the key - It's the fortieth, so pure, that's the right", role='assistant', function_call=None, tool_calls=None)


### ขั้นตอนที่ 3.2: โหลดและทดสอบโมเดลที่ปรับแต่งแล้วใน Playground

ตอนนี้คุณสามารถทดสอบโมเดลที่ปรับแต่งแล้วได้สองวิธี วิธีแรกคือเข้าไปที่ Playground แล้วใช้เมนูดรอปดาวน์ของ Models เพื่อเลือกโมเดลที่คุณเพิ่งปรับแต่งจากตัวเลือกที่มีอยู่ อีกวิธีหนึ่งคือใช้ตัวเลือก "Playground" ที่แสดงในแผง Fine-tuning (ดูภาพหน้าจอด้านบน) ซึ่งจะเปิดมุมมอง _เปรียบเทียบ_ ที่แสดงเวอร์ชันของโมเดลพื้นฐานและโมเดลที่ปรับแต่งแล้วแบบเคียงข้างกัน เพื่อให้คุณประเมินผลได้อย่างรวดเร็ว

![Fine-tuning job status](../../../../../translated_images/fine-tuned-playground-compare.56e06f0ad8922016497d39ced3d84ea296eec89073503f2bf346ec9718f913b5.th.png)

เพียงแค่กรอก system context ที่ใช้ในข้อมูลฝึกของคุณ และใส่คำถามทดสอบ คุณจะเห็นว่าทั้งสองฝั่งจะอัปเดตด้วย context และคำถามเดียวกัน จากนั้นรันการเปรียบเทียบ แล้วคุณจะเห็นความแตกต่างของผลลัพธ์ระหว่างทั้งสองโมเดล _สังเกตว่าโมเดลที่ปรับแต่งแล้วจะตอบกลับในรูปแบบที่คุณกำหนดไว้ในตัวอย่าง ขณะที่โมเดลพื้นฐานจะตอบตาม system prompt_

![Fine-tuning job status](../../../../../translated_images/fine-tuned-playground-launch.5a26495c983c6350c227e05700a47a89002d132949a56fa4ff37f266ebe997b2.th.png)

คุณจะเห็นว่าการเปรียบเทียบนี้ยังแสดงจำนวนโทเคนของแต่ละโมเดล และเวลาที่ใช้ในการประมวลผลด้วย **ตัวอย่างนี้เป็นเพียงตัวอย่างง่าย ๆ เพื่อแสดงขั้นตอนเท่านั้น ไม่ได้สะท้อนข้อมูลหรือสถานการณ์จริง** คุณอาจสังเกตได้ว่าทั้งสองตัวอย่างมีจำนวนโทเคนเท่ากัน (system context และ user prompt เหมือนกัน) แต่โมเดลที่ปรับแต่งแล้วจะใช้เวลาประมวลผลนานกว่า (เพราะเป็นโมเดล custom)

ในสถานการณ์จริง คุณจะไม่ได้ใช้ตัวอย่างง่าย ๆ แบบนี้ แต่จะปรับแต่งโมเดลกับข้อมูลจริง (เช่น แคตตาล็อกสินค้าเพื่อบริการลูกค้า) ซึ่งคุณภาพของคำตอบจะเห็นได้ชัดเจนมากขึ้น ใน _บริบท_ นั้น การจะได้คุณภาพคำตอบที่ใกล้เคียงกันจากโมเดลพื้นฐานจะต้องใช้การออกแบบ prompt ที่ซับซ้อนมากขึ้น ซึ่งจะทำให้ใช้โทเคนมากขึ้น และอาจใช้เวลาในการประมวลผลนานขึ้นด้วย _หากอยากลองใช้งานจริง สามารถดูตัวอย่างการปรับแต่งโมเดลใน OpenAI Cookbook ได้เลย_



---

**ข้อจำกัดความรับผิดชอบ**:  
เอกสารฉบับนี้ได้รับการแปลโดยใช้บริการแปลภาษา AI [Co-op Translator](https://github.com/Azure/co-op-translator) แม้ว่าเราจะพยายามอย่างเต็มที่เพื่อความถูกต้อง แต่โปรดทราบว่าการแปลโดยอัตโนมัติอาจมีข้อผิดพลาดหรือความไม่ถูกต้อง เอกสารต้นฉบับในภาษาต้นทางควรถือเป็นแหล่งข้อมูลที่เชื่อถือได้ สำหรับข้อมูลที่มีความสำคัญ แนะนำให้ใช้บริการแปลโดยนักแปลมืออาชีพ ทางเราจะไม่รับผิดชอบต่อความเข้าใจผิดหรือการตีความที่คลาดเคลื่อนซึ่งเกิดจากการใช้การแปลนี้
